In [7]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from io import BytesIO
import psycopg2

In [8]:
conn = psycopg2.connect(
            dbname="Hotel_Chain_Bookings",
            host="localhost",
            port="5432"
        )

# OPTUNA for model by hotels

In [9]:
df = pd.read_sql("select * from bookings_data\
                             where target is not null;", conn)

df['date_book'] = pd.to_datetime(df['date_book'])
df = df[['object_id', 'date_book', 'room_type_agg_id', 'target', 'working_day',
       'week_trend','tuesday', 'wednesday', 'thursday', 'friday',
       'saturday', 'sunday', 'month_trend', 'year_trend','feb', 'mar',
       'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', 't_mean',
       't_min', 't_max', 't_sd', 'u_mean', 'u_min', 'u_max', 'u_sd', 'ff_mean',
       'ff_min', 'ff_max', 'ff_sd', 'n_mean', 'n_min', 'n_max', 'n_sd',
       'nh_mean', 'nh_min', 'nh_max', 'nh_sd', 'h_mean', 'h_min', 'h_max',
       'h_sd', 'vv_mean', 'vv_min', 'vv_max', 'vv_sd', 'td_mean', 'td_min',
       'td_max', 'td_sd'
       ]]

df

/var/folders/k8/vyp2wdhd35d2sdxsqtdyxg180000gn/T/ipykernel_40931/25353974.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("select * from bookings_data\


,object_id,date_book,room_type_agg_id,target,working_day,week_trend,tuesday,wednesday,thursday,friday,...,h_max,h_sd,vv_mean,vv_min,vv_max,vv_sd,td_mean,td_min,td_max,td_sd
0,1,2019-01-01,1,1,False,1.0,True,False,False,False,...,2500.0,951.69,39.88,4.0,50.0,18.98,-3.65,-8.3,1.1,2.84
1,1,2019-01-01,2,1,False,1.0,True,False,False,False,...,2500.0,951.69,39.88,4.0,50.0,18.98,-3.65,-8.3,1.1,2.84
2,1,2019-01-01,3,2,False,1.0,True,False,False,False,...,2500.0,951.69,39.88,4.0,50.0,18.98,-3.65,-8.3,1.1,2.84
3,1,2019-01-02,2,1,False,2.0,False,True,False,False,...,300.0,51.75,15.13,2.0,50.0,21.54,-1.36,-4.5,-0.1,1.68
4,1,2019-01-02,3,11,False,2.0,False,True,False,False,...,300.0,51.75,15.13,2.0,50.0,21.54,-1.36,-4.5,-0.1,1.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19710,6,2024-10-28,4,2,True,0.0,False,False,False,False,...,2500.0,671.75,27.75,5.0,50.0,23.79,-3.65,-6.3,-1.8,1.68
19711,6,2024-10-29,3,4,True,1.0,True,False,False,False,...,2500.0,874.95,50.00,50.0,50.0,0.00,-4.25,-5.5,-3.2,0.81
19712,6,2024-10-29,4,3,True,1.0,True,False,False,False,...,2500.0,874.95,50.00,50.0,50.0,0.00,-4.25,-5.5,-3.2,0.81
19713,6,2024-10-30,3,11,True,2.0,False,True,False,False,...,2500.0,738.60,38.88,5.0,50.0,20.60,-3.41,-5.1,-1.8,1.09


In [10]:
agg_df = df.groupby(['object_id', 'date_book']).agg({'target': 'sum'}).reset_index().merge(df[['object_id', 'date_book','working_day',
       'week_trend','tuesday', 'wednesday', 'thursday', 'friday',
       'saturday', 'sunday', 'month_trend', 'year_trend','feb', 'mar',
       'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', 't_mean',
       't_min', 't_max', 't_sd', 'u_mean', 'u_min', 'u_max', 'u_sd', 'ff_mean',
       'ff_min', 'ff_max', 'ff_sd', 'n_mean', 'n_min', 'n_max', 'n_sd',
       'nh_mean', 'nh_min', 'nh_max', 'nh_sd', 'h_mean', 'h_min', 'h_max',
       'h_sd', 'vv_mean', 'vv_min', 'vv_max', 'vv_sd', 'td_mean', 'td_min',
       'td_max', 'td_sd'
       ]].drop_duplicates(), on=['object_id', 'date_book'], how = 'left')

agg_df.rename(columns={'date_book': 'ds', 'target': 'y'}, inplace=True)
agg_df

,object_id,ds,y,working_day,week_trend,tuesday,wednesday,thursday,friday,saturday,...,h_max,h_sd,vv_mean,vv_min,vv_max,vv_sd,td_mean,td_min,td_max,td_sd
0,1,2019-01-01,4,False,1.0,True,False,False,False,False,...,2500.0,951.69,39.88,4.0,50.0,18.98,-3.65,-8.3,1.1,2.84
1,1,2019-01-02,12,False,2.0,False,True,False,False,False,...,300.0,51.75,15.13,2.0,50.0,21.54,-1.36,-4.5,-0.1,1.68
2,1,2019-01-03,21,False,3.0,False,False,True,False,False,...,300.0,75.59,14.50,4.0,50.0,16.93,-6.51,-11.1,-4.7,1.99
3,1,2019-01-04,13,False,4.0,False,False,False,True,False,...,2500.0,1077.03,33.50,4.0,50.0,20.89,-9.86,-12.3,-8.0,1.78
4,1,2019-01-05,16,False,5.0,False,False,False,False,True,...,2500.0,715.02,14.38,4.0,50.0,17.12,-7.35,-8.7,-5.8,1.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10941,6,2024-10-26,5,False,5.0,False,False,False,False,True,...,2500.0,959.07,45.00,10.0,50.0,14.14,-4.94,-7.2,-3.3,1.26
10942,6,2024-10-27,6,False,6.0,False,False,False,False,False,...,2500.0,694.37,50.00,50.0,50.0,0.00,-6.95,-8.3,-6.0,0.68
10943,6,2024-10-28,6,True,0.0,False,False,False,False,False,...,2500.0,671.75,27.75,5.0,50.0,23.79,-3.65,-6.3,-1.8,1.68
10944,6,2024-10-29,7,True,1.0,True,False,False,False,False,...,2500.0,874.95,50.00,50.0,50.0,0.00,-4.25,-5.5,-3.2,0.81


In [11]:
df_catboost = agg_df.copy()
df_catboost['Lag_1'] = df_catboost.groupby('object_id')['y'].shift(1)
df_catboost['Lag_7'] = df_catboost.groupby('object_id')['y'].shift(7)
df_catboost['Lag_30'] = df_catboost.groupby('object_id')['y'].shift(30)
df_catboost['Lag_365'] = df_catboost.groupby('object_id')['y'].shift(365)

df_catboost.set_index('ds', inplace=True)
df_catboost

,object_id,y,working_day,week_trend,tuesday,wednesday,thursday,friday,saturday,sunday,...,vv_max,vv_sd,td_mean,td_min,td_max,td_sd,Lag_1,Lag_7,Lag_30,Lag_365
ds,,,,,,,,,,,,,,,,,,,,,
2019-01-01,1,4,False,1.0,True,False,False,False,False,False,...,50.0,18.98,-3.65,-8.3,1.1,2.84,NaN,NaN,NaN,NaN
2019-01-02,1,12,False,2.0,False,True,False,False,False,False,...,50.0,21.54,-1.36,-4.5,-0.1,1.68,4.0,NaN,NaN,NaN
2019-01-03,1,21,False,3.0,False,False,True,False,False,False,...,50.0,16.93,-6.51,-11.1,-4.7,1.99,12.0,NaN,NaN,NaN
2019-01-04,1,13,False,4.0,False,False,False,True,False,False,...,50.0,20.89,-9.86,-12.3,-8.0,1.78,21.0,NaN,NaN,NaN
2019-01-05,1,16,False,5.0,False,False,False,False,True,False,...,50.0,17.12,-7.35,-8.7,-5.8,1.16,13.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-26,6,5,False,5.0,False,False,False,False,True,False,...,50.0,14.14,-4.94,-7.2,-3.3,1.26,9.0,4.0,11.0,9.0
2024-10-27,6,6,False,6.0,False,False,False,False,False,True,...,50.0,0.00,-6.95,-8.3,-6.0,0.68,5.0,10.0,8.0,13.0
2024-10-28,6,6,True,0.0,False,False,False,False,False,False,...,50.0,23.79,-3.65,-6.3,-1.8,1.68,6.0,13.0,10.0,17.0


In [24]:
%pip install catboost

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 27.1 MB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 14.8 MB 613 kB/s eta 0:00:01
     |████████████████████████████████| 47 kB 698 kB/s eta 0:00:01
     |████████████████████████████████| 30.3 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 7.8 MB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 14.0 MB 1.6 MB/s eta 0:00:01
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
     |████████████████████████████████| 2.8 MB 775 kB/s eta 0:00:01
     |████████████████████████████████| 64 kB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 249 kB 994 kB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 322 kB 1.0 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing 

In [3]:
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 478.1 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [21]:
np.random.seed(32)

df_catboost['object_id'] = df_catboost['object_id'].astype('category')

X = df_catboost.drop('y', axis=1)
y = df_catboost['y']

test_start_date = '2024-07-29'

X_train = X[X.index < test_start_date]
y_train = y[y.index < test_start_date]
X_test = X[X.index >= test_start_date]
y_test = y[y.index >= test_start_date]

In [ ]:
import optuna
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import joblib
import os

def calculate_mape(y_true, y_pred):
    """Calculate Mean Absolute Percentage Error (MAPE)"""
    return np.mean(np.abs((y_true - y_pred)) / y_true)

def objective(trial, X_train, y_train, X_test, y_test):
    cat_features = list(X_train.select_dtypes(include=['category', 'object']).columns)
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.01), 
        'depth': trial.suggest_int('depth', 4, 8),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-1, 10), 
        'random_strength': trial.suggest_float('random_strength', 1e-1, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'od_type': 'Iter',
        'od_wait': trial.suggest_int('od_wait', 10, 50),
        'cat_features': cat_features,
        'verbose': False,
        'random_state': 42,
        'loss_function': 'MAPE',
        'eval_metric': 'MAPE',
        'custom_metric': ['RMSE']
    }

    model = CatBoostRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=(X_test, y_test),
        early_stopping_rounds=50,
        verbose=False
    )

    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = calculate_mape(y_test, y_pred)

    trial.set_user_attr('rmse', rmse)
    trial.set_user_attr('mape', mape)
    trial.set_user_attr('model', model)  
    
    return rmse  

os.makedirs('saved_models', exist_ok=True)

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=500)

results = []
for trial in study.trials:
    results.append({
        'params': trial.params,
        'RMSE': trial.user_attrs['rmse'],
        'MAPE': trial.user_attrs['mape'],
        'state': trial.state.name
    })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='MAPE').reset_index(drop=True)

best_model = study.best_trial.user_attrs['model']

print("\nBest trial:")
best_trial = study.best_trial
print(f"  RMSE: {best_trial.user_attrs['rmse']:.4f}")
print(f"  MAPE: {best_trial.user_attrs['mape']:.2f}")
print("  Params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

print("\nTop 5 trials:")
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print(results_df.head(5))

[I 2025-04-06 13:04:01,854] A new study created in memory with name: no-name-9602a8c2-8f45-46ed-9030-82578a4cac49
[I 2025-04-06 13:04:03,249] Trial 0 finished with value: 4.588881880635587 and parameters: {'iterations': 332, 'learning_rate': 0.00902902343588478, 'depth': 5, 'l2_leaf_reg': 2.969091296584154, 'random_strength': 9.503200420671451, 'bagging_temperature': 0.35853669956460665, 'od_wait': 14}. Best is trial 0 with value: 4.588881880635587.
[I 2025-04-06 13:04:05,676] Trial 1 finished with value: 4.682090411239104 and parameters: {'iterations': 956, 'learning_rate': 0.0025683661527088546, 'depth': 6, 'l2_leaf_reg': 3.014491252955073, 'random_strength': 7.009266396974382, 'bagging_temperature': 0.12526810816761025, 'od_wait': 34}. Best is trial 0 with value: 4.588881880635587.
[I 2025-04-06 13:04:07,365] Trial 2 finished with value: 4.487338255666896 and parameters: {'iterations': 679, 'learning_rate': 0.004383029488703427, 'depth': 6, 'l2_leaf_reg': 5.190961460628077, 'random_


Best trial:
  RMSE: 4.0484
  MAPE: 0.48
  Params: 
    iterations: 723
    learning_rate: 0.009690956330283655
    depth: 4
    l2_leaf_reg: 1.189939623282865
    random_strength: 8.021584887054352
    bagging_temperature: 0.06349080858678205
    od_wait: 29

Top 5 trials:
                                              params      RMSE      MAPE     state
0  {'iterations': 723, 'learning_rate': 0.0096909...  4.048391  0.475728  COMPLETE
1  {'iterations': 824, 'learning_rate': 0.0097934...  4.070090  0.475865  COMPLETE
2  {'iterations': 701, 'learning_rate': 0.0099962...  4.072015  0.478595  COMPLETE
3  {'iterations': 800, 'learning_rate': 0.0098401...  4.074719  0.475679  COMPLETE
4  {'iterations': 735, 'learning_rate': 0.0091543...  4.075389  0.477447  COMPLETE


In [33]:
results_df

,params,RMSE,MAPE,state
0,"{'iterations': 723, 'learning_rate': 0.0096909...",4.048391,0.475728,COMPLETE
1,"{'iterations': 824, 'learning_rate': 0.0097934...",4.070090,0.475865,COMPLETE
2,"{'iterations': 701, 'learning_rate': 0.0099962...",4.072015,0.478595,COMPLETE
3,"{'iterations': 800, 'learning_rate': 0.0098401...",4.074719,0.475679,COMPLETE
4,"{'iterations': 735, 'learning_rate': 0.0091543...",4.075389,0.477447,COMPLETE
...,...,...,...,...
495,"{'iterations': 763, 'learning_rate': 0.0012434...",5.777032,0.559170,COMPLETE
496,"{'iterations': 792, 'learning_rate': 0.0010535...",5.846278,0.564810,COMPLETE
497,"{'iterations': 109, 'learning_rate': 0.0079217...",5.913965,0.561911,COMPLETE
498,"{'iterations': 100, 'learning_rate': 0.0072682...",6.241160,0.598074,COMPLETE


In [ ]:
model_path = 'saved_models/best_catboost_model_hotel.cbm'
best_model.save_model(model_path)
print(f"\nBest model saved to {model_path}")


Best model saved to saved_models/best_catboost_model_hotel.cbm


In [ ]:
preds = best_model.predict(X_test)
results = pd.DataFrame({'yhat': preds, 'y': y_test})
results['object_id'] = X_test['object_id'].values
results.reset_index().to_excel('catboost_hotel_optuna.xlsx')

# OPTUNA for model by room types

In [64]:
df_catboost_room = df.copy()
df_catboost_room['Lag_1'] = df_catboost_room.groupby(['object_id', 'room_type_agg_id'])['target'].shift(1)
df_catboost_room['Lag_7'] = df_catboost_room.groupby(['object_id', 'room_type_agg_id'])['target'].shift(7)
df_catboost_room['Lag_30'] = df_catboost_room.groupby(['object_id', 'room_type_agg_id'])['target'].shift(30)
df_catboost_room['Lag_365'] = df_catboost_room.groupby(['object_id', 'room_type_agg_id'])['target'].shift(365)
df_catboost_room.rename(columns={'date_book': 'ds', 'target': 'y'}, inplace=True)
df_catboost_room.set_index('ds', inplace=True)
df_catboost_room

,object_id,room_type_agg_id,y,working_day,week_trend,tuesday,wednesday,thursday,friday,saturday,sunday,month_trend,year_trend,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,t_mean,t_min,t_max,t_sd,u_mean,u_min,u_max,u_sd,ff_mean,ff_min,ff_max,ff_sd,n_mean,n_min,n_max,n_sd,nh_mean,nh_min,nh_max,nh_sd,h_mean,h_min,h_max,h_sd,vv_mean,vv_min,vv_max,vv_sd,td_mean,td_min,td_max,td_sd,Lag_1,Lag_7,Lag_30,Lag_365
ds,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-01,1,1,1,False,1.0,True,False,False,False,False,False,1.0,1.0,False,False,False,False,False,False,False,False,False,False,False,-2.44,-5.7,1.6,2.30,91.63,82.0,97.0,4.75,3.50,2.0,5.0,1.07,77.50,0.0,100.0,34.95,57.14,20.0,100.0,33.52,1171.43,300.0,2500.0,951.69,39.88,4.0,50.0,18.98,-3.65,-8.3,1.1,2.84,NaN,NaN,NaN,NaN
2019-01-01,1,2,1,False,1.0,True,False,False,False,False,False,1.0,1.0,False,False,False,False,False,False,False,False,False,False,False,-2.44,-5.7,1.6,2.30,91.63,82.0,97.0,4.75,3.50,2.0,5.0,1.07,77.50,0.0,100.0,34.95,57.14,20.0,100.0,33.52,1171.43,300.0,2500.0,951.69,39.88,4.0,50.0,18.98,-3.65,-8.3,1.1,2.84,NaN,NaN,NaN,NaN
2019-01-01,1,3,2,False,1.0,True,False,False,False,False,False,1.0,1.0,False,False,False,False,False,False,False,False,False,False,False,-2.44,-5.7,1.6,2.30,91.63,82.0,97.0,4.75,3.50,2.0,5.0,1.07,77.50,0.0,100.0,34.95,57.14,20.0,100.0,33.52,1171.43,300.0,2500.0,951.69,39.88,4.0,50.0,18.98,-3.65,-8.3,1.1,2.84,NaN,NaN,NaN,NaN
2019-01-02,1,2,1,False,2.0,False,True,False,False,False,False,2.0,1.0,False,False,False,False,False,False,False,False,False,False,False,-0.70,-3.7,0.5,1.58,95.50,94.0,96.0,0.93,1.88,0.0,4.0,1.73,100.00,100.0,100.0,0.00,100.00,100.0,100.0,0.00,237.50,200.0,300.0,51.75,15.13,2.0,50.0,21.54,-1.36,-4.5,-0.1,1.68,1.0,NaN,NaN,NaN
2019-01-02,1,3,11,False,2.0,False,True,False,False,False,False,2.0,1.0,False,False,False,False,False,False,False,False,False,False,False,-0.70,-3.7,0.5,1.58,95.50,94.0,96.0,0.93,1.88,0.0,4.0,1.73,100.00,100.0,100.0,0.00,100.00,100.0,100.0,0.00,237.50,200.0,300.0,51.75,15.13,2.0,50.0,21.54,-1.36,-4.5,-0.1,1.68,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-28,6,4,2,True,0.0,False,False,False,False,False,False,28.0,10.0,False,False,False,False,False,False,False,False,True,False,False,-1.48,-2.2,-0.5,0.50,85.75,71.0,93.0,9.13,2.00,2.0,2.0,0.00,81.25,20.0,100.0,29.49,76.25,20.0,100.0,36.23,2262.50,600.0,2500.0,671.75,27.75,5.0,50.0,23.79,-3.65,-6.3,-1.8,1.68,2.0,4.0,5.0,6.0
2024-10-29,6,3,4,True,1.0,True,False,False,False,False,False,29.0,10.0,False,False,False,False,False,False,False,False,True,False,False,-2.70,-4.4,-1.4,0.96,89.25,83.0,94.0,3.88,1.13,1.0,2.0,0.35,96.25,90.0,100.0,5.18,68.75,10.0,100.0,36.43,1462.50,600.0,2500.0,874.95,50.00,50.0,50.0,0.00,-4.25,-5.5,-3.2,0.81,4.0,5.0,9.0,7.0
2024-10-29,6,4,3,True,1.0,True,False,False,False,False,False,29.0,10.0,False,False,False,False,False,False,False,False,True,False,False,-2.70,-4.4,-1.4,0.96,89.25,83.0,94.0,3.88,1.13,1.0,2.0,0.35,96.25,90.0,100.0,5.18,68.75,10.0,100.0,36.43,1462.50,600.0,2500.0,874.95,50.00,50.0,50.0,0.00,-4.25,-5.5,-3.2,0.81,2.0,7.0,2.0,4.0


In [65]:
df_catboost_room['object_id'] = df_catboost_room['object_id'].astype('category')
df_catboost_room['room_type_agg_id'] = df_catboost_room['room_type_agg_id'].astype('category')

In [66]:
X = df_catboost_room.drop('y', axis=1)
y = df_catboost_room['y']

test_start_date = '2024-07-29'

X_train = X[X.index < test_start_date]
y_train = y[y.index < test_start_date]
X_test = X[X.index >= test_start_date]
y_test = y[y.index >= test_start_date]

In [ ]:

def objective(trial, X_train, y_train, X_test, y_test):
    cat_features = list(X_train.select_dtypes(include=['category', 'object']).columns)
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.01), 
        'depth': trial.suggest_int('depth', 4, 8),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-1, 10), 
        'random_strength': trial.suggest_float('random_strength', 1e-1, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'od_type': 'Iter',
        'od_wait': trial.suggest_int('od_wait', 10, 50),
        'cat_features': cat_features,
        'verbose': False,
        'random_state': 42,
        'loss_function': 'MAPE',
        'eval_metric': 'MAPE',
        'custom_metric': ['RMSE']
    }

    model = CatBoostRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=(X_test, y_test),
        early_stopping_rounds=50,
        verbose=False
    )

    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = calculate_mape(y_test, y_pred)

    trial.set_user_attr('rmse', rmse)
    trial.set_user_attr('mape', mape)
    trial.set_user_attr('model', model)
    
    return rmse 

os.makedirs('saved_models', exist_ok=True)

study_room = optuna.create_study(direction='minimize')
study_room.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=500)

results_room = []
for trial in study_room.trials:
    results_room.append({
        'params': trial.params,
        'RMSE': trial.user_attrs['rmse'],
        'MAPE': trial.user_attrs['mape'],
        'state': trial.state.name
    })

results_df_room = pd.DataFrame(results_room)
results_df_room = results_df_room.sort_values(by='MAPE').reset_index(drop=True)

best_model_room = study_room.best_trial.user_attrs['model']

print("\nBest trial:")
best_trial_room = study_room.best_trial
print(f"  RMSE: {best_trial_room.user_attrs['rmse']:.4f}")
print(f"  MAPE: {best_trial_room.user_attrs['mape']:.2f}")
print("  Params: ")
for key, value in best_trial_room.params.items():
    print(f"    {key}: {value}")

print("\nTop 5 trials:")
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print(results_df_room.head(5))

[I 2025-04-06 15:27:19,414] A new study created in memory with name: no-name-3cfcf9b5-8437-4777-83f7-fc1055a95c88
[I 2025-04-06 15:27:21,406] Trial 0 finished with value: 4.397139349355465 and parameters: {'iterations': 504, 'learning_rate': 0.0018180667040379095, 'depth': 6, 'l2_leaf_reg': 4.111981659612862, 'random_strength': 7.8153112469658454, 'bagging_temperature': 0.6804557846381688, 'od_wait': 48}. Best is trial 0 with value: 4.397139349355465.
[I 2025-04-06 15:27:23,037] Trial 1 finished with value: 4.027303039690926 and parameters: {'iterations': 727, 'learning_rate': 0.003569320775171445, 'depth': 6, 'l2_leaf_reg': 3.1383196746127777, 'random_strength': 1.9982880146607038, 'bagging_temperature': 0.11393273775365409, 'od_wait': 11}. Best is trial 1 with value: 4.027303039690926.
[I 2025-04-06 15:27:23,568] Trial 2 finished with value: 4.692215549704017 and parameters: {'iterations': 213, 'learning_rate': 0.002841311965038046, 'depth': 4, 'l2_leaf_reg': 2.334534677805539, 'rand


Best trial:
  RMSE: 3.7664
  MAPE: 0.50
  Params: 
    iterations: 962
    learning_rate: 0.006753501900406873
    depth: 8
    l2_leaf_reg: 4.9032323813321215
    random_strength: 8.686093767695617
    bagging_temperature: 0.9790981045849421
    od_wait: 24

Top 5 trials:
                                              params      RMSE      MAPE     state
0  {'iterations': 997, 'learning_rate': 0.0092482...  3.946950  0.490192  COMPLETE
1  {'iterations': 924, 'learning_rate': 0.0069873...  3.911698  0.490826  COMPLETE
2  {'iterations': 985, 'learning_rate': 0.0090748...  3.885078  0.491094  COMPLETE
3  {'iterations': 877, 'learning_rate': 0.0067466...  3.888545  0.491328  COMPLETE
4  {'iterations': 957, 'learning_rate': 0.0063204...  3.866648  0.491588  COMPLETE


In [ ]:
preds_room = best_model_room.predict(X_test)
results_room = pd.DataFrame({'yhat': preds_room, 'y': y_test})
results_room['object_id'] = X_test['object_id'].values
results_room['room_type_agg_id'] = X_test['room_type_agg_id'].values
results_room.reset_index().to_excel('catboost_hotel_room_optuna.xlsx')

In [70]:
model_path_room = 'saved_models/best_catboost_model_hotel_room.cbm'
best_model_room.save_model(model_path_room)
print(f"\nBest model saved to {model_path}")


Best model saved to saved_models/best_catboost_model_hotel.cbm
